# Task Description

Comparison of the musical behavior and preferences between Moscow and Saint-Petersburg users based on Yandex.Music data.

### Targets of the research - check 3 hypotheses
* The activity of the users depends on the day of the week and differs in Moscow and Saint-Petersburg;
* The genres preferences differ in the Monday mornings and Friday evenings in the cities;
* Moscow dwellers enjoy listening to the pop music meanwhile in Saint-Petersburg russian rap is more popular.


# Stage 1. Data obtain

Getting familiar with the dataset that was provided by Yandex.

In [4]:
import pandas as pd

df = pd.read_csv("D:\Yandex. Data Analysis\Project_1\yandex_music_project.csv")

Display the first 10 raws of the dataset

In [5]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Let's take a closer look at the table.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


There are 7 columns in the table with data type 'object'.

Description of the columns' names:
* userID — User identification number;
* Track — Soundtrack name;
* artist — Name of the artist;
* genre — Name of the genre;
* City — City where the track has been listened;
* time — Time when the track has been listened;
* Day — Day of the week.

## Conclusion

The number of values differs in the columns. It means that some data are missing in the table. Here are 2 issues that have to be solved: gaps and poor-quality column names. In order to check the hypotheses the most valuable columns are 'time', 'day' and 'city'. The information in the 'genre' column will help to find out the most populars genres in the cities.

# Stage 2. Data preprocessing

Let's eliminate the gaps, rename the columns, and also check the data for duplicates. Here are the names of the columns:

In [7]:
display(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

There are some spaces and different cases in the names that can make it difficult to access the data. We will rename the headers of the columns.

In [9]:
df = df.rename(columns = {'  userID':'user_id',
'Track':'track',
'  City  ':'city',
'Day':'day'})
display(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Let's check now if there are gaps in the table.

In [10]:
df_null = df.isna().sum()
display(df_null)

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

The null values mean that some soundtracks do not have full information. The reasons may be different: for example, a specific performer of a song has not been mentioned. Worse, if there are problems with writing data. Each individual case must be analyzed and the cause identified.

We replace the missing values in the columns with the name of the track and the artist with the string 'unknown'.

In [11]:
columns_to_replace = df
for value in columns_to_replace:
    columns_to_replace['track'] = columns_to_replace['track'].fillna('unknown')
    columns_to_replace['artist'] = columns_to_replace['artist'].fillna('unknown')
    columns_to_replace['genre'] = columns_to_replace['genre'].fillna('unknown')
    
display(columns_to_replace)

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
...,...,...,...,...,...,...,...
65074,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
65075,D08D4A55,Maybe One Day (feat. Black Spade),Blu & Exile,hip,Saint-Petersburg,10:00:00,Monday
65076,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
65077,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


Make sure that there are no more gaps left:

In [12]:
columns_to_replace_null = columns_to_replace.isna().sum()
display(columns_to_replace_null)

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

It's also necessary to check whether there are any duplicates. If so we have to delete them.

In [13]:
display(columns_to_replace.duplicated().sum())

3826

In [15]:
columns_to_replace = columns_to_replace.drop_duplicates().reset_index(drop=True)
display(columns_to_replace.duplicated().sum())

0

Now we have to get rid of the implicit duplicates in the 'genre' column. Let's see how many unique values there are in that column.

In [16]:
unique_genres = columns_to_replace['genre'].sort_values().unique()
display(unique_genres)

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

As it can be seen here are the following implicit duplicates:
* hip
* hop
* hip-hop

In order to clean up the table from these duplicates we will create a function. It will correct wrong values (implicit duplicates) the 'genre' column  with the correct one - 'hiphop'. 

In [17]:
def replace_wrong_genres(wrong_genres,correct_genre):
    for wrong_genre in wrong_genres:
       columns_to_replace['genre'] = columns_to_replace['genre'].replace(wrong_genre, correct_genre)
    
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'
replace_wrong_genres(wrong_genres, correct_genre)

Let's make it sure that there are no more duplicates.

In [18]:
unique_genres = columns_to_replace['genre'].sort_values().unique()
display(unique_genres)

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

## Conclusion

During the data preprocessing stage gaps and inaccurate column names as well as different types of duplicates were found.
These things made it difficult to conduct a proper data analysis so they were treated in an appropriate manner. The information about the genres is vital for the futher analysis so the gap values are replaced with placeholders 'uknown'. Now the corrected dataset is ready for work.

## Hypothesis №1 - The activity of the users depends on the day of the week and differs in Moscow and Saint-Petersburg;

For each city we set the number of songs with a known genre listened on these days and compare the results.

In [19]:
df = columns_to_replace # Let's кename the corrected table to the default variable
df.groupby('city')['time'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

The number of listeners in Moscow is bigger than in Saint-Petersburg. However it doesn't mean that the users are more active in Moscow. Let's group the data regarding the day of the week.

In [20]:
df.groupby('day')['city'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64

Now we create a function that returns a number of tracks that were listened in each exact day. We will apply this function to each city regarding every certain day (Monday, Wednesday and Friday).

In [21]:
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [22]:
number_tracks("Monday", "Moscow")

15740

In [23]:
number_tracks("Monday", "Saint-Petersburg")

5614

In [24]:
number_tracks("Wednesday", "Moscow")

11056

In [25]:
number_tracks("Wednesday", "Saint-Petersburg")

7003

In [26]:
number_tracks("Friday", "Moscow")

15945

In [27]:
number_tracks("Friday", "Saint-Petersburg")

5895

In [29]:
columns = ['City', 'Monday', 'Wednesday', 'Friday']
data = [['Moscow', 16715, 11755, 16890], ['Saint-Petersburg', 5982, 7478, 6259]]
pd.DataFrame(data=data, columns=columns)

,City,Monday,Wednesday,Friday
0,Moscow,16715,11755,16890
1,Saint-Petersburg,5982,7478,6259


## Conclusion
As it can be seen in the last table the Moscow citizens are more active on Mondays and Fridays whereas in Saint-Petersburg they are on Wednesdays. Hypothesis №1 is approved.

## Hypothesis №2 - the genres preferences differ in the Monday mornings and Friday evenings in the cities

Let's filter the table regarding every city.

In [30]:
moscow_general = df.loc[df.loc[:,'city'] == 'Moscow']
display(moscow_general.head(5))

,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday


In [31]:
spb_general = df.loc[df.loc[:,'city'] == 'Saint-Petersburg']
display(spb_general.head(5))

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday


Now we create a function that will return information about top-10 genres which were listened in an exact day, time and city.

In [32]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return genre_df_sorted.head(10)

In [33]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [34]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [35]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [36]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

## Conclusion

Popular genres on Monday morning in St. Petersburg and Moscow turned out to be similar: everywhere, as expected, pop is popular. Despite this, the ending of the top 10 for the two cities is different: in St. Petersburg there are jazz and Russian rap, meanwhile in Moscow it is the world genre. At the end of the week, the situation does not change. Pop music is still in the first place. Thus, the second hypothesis was only partially confirmed:
* Users listen to similar music at the beginning of the week and at the end;
* The difference between Moscow and St. Petersburg is not noticable.

## Hypothesis №3 - Moscow dwellers enjoy listening to the pop music meanwhile in Saint-Petersburg russian rap is more popular

Let's group the 'moscow_general' table by genre, count the number of compositions of each genre.

In [37]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)
display(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Let's do the same procedure with Sain-Petersburg.

In [38]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)
display(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

## Conclusion

The hypothesis was partially confirmed:

* Pop music is the most popular genre in Moscow, as the hypothesis suggested. Moreover, in the top 10 genres there is a similar genre - Russian popular music.
* Contrary to expectations, rap is equally popular in Moscow and St. Petersburg.

# Overall conclusion

1. The day of the week affects the activity of users in Moscow and St. Petersburg in different ways.
The first hypothesis was fully confirmed.

2. Musical preferences do not change much during the week - be it Moscow or St. Petersburg. Small differences are noticeable at the beginning of the week, on Mondays:
* in Moscow they listen to music of the “world” genre,
* Petersburg - jazz and classical music.
Thus, the second hypothesis was only partly confirmed. This result could have been different were it not for gaps in the data.

3. The tastes of users in Moscow and St. Petersburg have more in common than differences. Contrary to expectations, genre preferences in St. Petersburg resemble those in Moscow.
The third hypothesis was not confirmed. If there are differences in preferences, they are invisible to the bulk of users.